In [4]:
import numpy as np
import pandas as pd
import requests
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

In [5]:
def fetch_binance_data(symbol, interval='1h', limit=1000):
    url = "https://api.binance.com/api/v3/klines"
    params = {'symbol': symbol, 'interval': interval, 'limit': limit}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['volume'] = df['volume'].astype(float)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    return df

In [6]:
def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def create_features(df):
    df['SMA_50'] = df['close'].rolling(window=50).mean()
    df['SMA_200'] = df['close'].rolling(window=200).mean()
    df['RSI'] = compute_rsi(df['close'], 14)
    df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['Bollinger_High'] = df['close'].rolling(window=20).mean() + 2 * df['close'].rolling(window=20).std()
    df['Bollinger_Low'] = df['close'].rolling(window=20).mean() - 2 * df['close'].rolling(window=20).std()
    df['volume_change'] = df['volume'].pct_change()
    
    # Lag features
    for i in range(1, 11):
        df[f'close_lag_{i}'] = df['close'].shift(i)
    
    df.dropna(inplace=True)  # Remove rows with NaN values

In [7]:
def create_feature_label_data(df):
    df['future_close'] = df['close'].shift(-7)  # Predict price 7 days ahead
    df.dropna(inplace=True)  # Remove rows with NaN values

    features = df[['SMA_50', 'SMA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'MACD_signal', 'Bollinger_High', 'Bollinger_Low', 'volume_change'] + [f'close_lag_{i}' for i in range(1, 11)]]
    labels = df['future_close']
    return features, labels

def build_and_train_model(features, labels):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    
    X_train, X_test, y_train, y_test = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Mean Squared Error: {mse:.2f}")
    print(f"Model Accuracy (R² score): {r2:.2%}")
    
    return model, scaler

In [8]:
def predict_future_prices(model, scaler, df, conversion_rate, symbol):
    today = df['close_time'].max()
    
    # Set next_midnight to midnight of the next day
    next_midnight = (today + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
    
    future_dates = pd.date_range(start=next_midnight, periods=168, freq='h')  # Updated freq to 'h'
    future_df = pd.DataFrame(index=future_dates)
    
    # Ensure features_count matches
    features_count = len(df[['SMA_50', 'SMA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'MACD_signal', 'Bollinger_High', 'Bollinger_Low', 'volume_change'] + [f'close_lag_{i}' for i in range(1, 11)]].columns)
    
    last_features = df[['SMA_50', 'SMA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'MACD_signal', 'Bollinger_High', 'Bollinger_Low', 'volume_change'] + [f'close_lag_{i}' for i in range(1, 11)]].iloc[-1].values
    last_features = np.reshape(last_features, (1, features_count))  # Reshape for prediction
    
    future_predictions = []
    
    for _ in range(168):  # Predict for the next 168 hours
        # Predict the price for the next hour
        scaled_last_features = scaler.transform(pd.DataFrame(last_features, columns=df[['SMA_50', 'SMA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'MACD_signal', 'Bollinger_High', 'Bollinger_Low', 'volume_change'] + [f'close_lag_{i}' for i in range(1, 11)]].columns))
        prediction = model.predict(scaled_last_features)
        future_predictions.append(prediction[0])
        
        # Update last_features for next prediction
        last_features = np.roll(last_features, shift=-1, axis=1)  # Roll features left
        last_features[0, -1] = prediction[0]  # Update last feature with the new prediction

    future_df['predicted_close'] = future_predictions
    future_df['predicted_close_inr'] = future_df['predicted_close'] * conversion_rate

    # Find best times to buy and sell
    min_index = future_df['predicted_close'].idxmin()
    max_index = future_df['predicted_close'].idxmax()

    best_time_to_buy = future_df.loc[min_index].name
    best_time_to_sell = future_df.loc[max_index].name
    best_price_to_buy = future_df.loc[min_index, 'predicted_close']
    best_price_to_sell = future_df.loc[max_index, 'predicted_close']
    best_price_to_buy_inr = best_price_to_buy * conversion_rate
    best_price_to_sell_inr = best_price_to_sell * conversion_rate

    # Format the datetime to 'YEAR-MONTH-DATE  HOUR:MINUTE PM/AM'
    best_time_to_buy = best_time_to_buy.strftime('%Y-%m-%d %I:%M %p')
    best_time_to_sell = best_time_to_sell.strftime('%Y-%m-%d %I:%M %p')

    future_df['formatted_date'] = future_df.index.strftime('%Y-%m-%d %I:%M %p')

    print(f"\nBest time to buy {symbol}: {best_time_to_buy} at price ${best_price_to_buy:.8f} ({best_price_to_buy_inr:.2f} INR)")
    print(f"Best time to sell {symbol}: {best_time_to_sell} at price ${best_price_to_sell:.8f} ({best_price_to_sell_inr:.2f} INR)")

    return future_df

In [9]:
def get_conversion_rate():
    url = "https://api.exchangerate-api.com/v4/latest/USD"
    response = requests.get(url)
    data = response.json()
    conversion_rate = data['rates'].get('INR', 1)  # Default to 1 if INR rate is not found
    return conversion_rate

In [10]:
def retrain_model(symbol):
    # Fetch the latest data
    df = fetch_binance_data(symbol)
    
    # Create features and labels
    create_features(df)
    features, labels = create_feature_label_data(df)
    
    # Build and train the model
    model, scaler = build_and_train_model(features, labels)
    
    return model, scaler, df

In [11]:
def main():
    symbol = input("Enter the cryptocurrency symbol (e.g., BTC): ")
    symbol = symbol + 'USDT'
    model, scaler, df = retrain_model(symbol)

    conversion_rate = get_conversion_rate()
    future_df = predict_future_prices(model, scaler, df, conversion_rate, symbol)

    # Print the predicted prices in a tabular format
    print("\nPredicted Prices for {} in the next 168 hours:".format(symbol))

    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 1000)
    
    print(future_df[['formatted_date', 'predicted_close', 'predicted_close_inr']].to_string(index=False, header=True, float_format=lambda x: '{:.8f}'.format(x)))

    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')

In [12]:
if __name__ == "__main__":
    main()

Model Mean Squared Error: 161519.03
Model Accuracy (R² score): 99.01%

Best time to buy BTCUSDT: 2024-08-03 02:00 AM at price $63361.70870000 (5306543.10 INR)
Best time to sell BTCUSDT: 2024-08-08 07:00 AM at price $68950.21170000 (5774580.23 INR)

Predicted Prices for BTCUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-08-03 12:00 AM   63572.36900000     5324185.90375000
2024-08-03 01:00 AM   63886.18870000     5350468.30362499
2024-08-03 02:00 AM   63361.70870000     5306543.10362500
2024-08-03 03:00 AM   65622.02630000     5495844.70262500
2024-08-03 04:00 AM   64303.49040000     5385417.32100000
2024-08-03 05:00 AM   63616.51400000     5327883.04750000
2024-08-03 06:00 AM   65980.61930000     5525876.86637500
2024-08-03 07:00 AM   65349.21210000     5472996.51337500
2024-08-03 08:00 AM   65502.06940000     5485798.31225000
2024-08-03 09:00 AM   65395.40460000     5476865.13525000
2024-08-03 10:00 AM   65431.90810000     5479922.30337500
202

In [13]:
if __name__ == "__main__":
    main()

Model Mean Squared Error: 635.60
Model Accuracy (R² score): 97.43%

Best time to buy ETHUSDT: 2024-08-03 01:00 AM at price $3037.94180000 (254427.63 INR)
Best time to sell ETHUSDT: 2024-08-07 12:00 AM at price $3420.68010000 (286481.96 INR)

Predicted Prices for ETHUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-08-03 12:00 AM    3066.23010000      256796.77087500
2024-08-03 01:00 AM    3037.94180000      254427.62575000
2024-08-03 02:00 AM    3196.86830000      267737.72012500
2024-08-03 03:00 AM    3254.90390000      272598.20162500
2024-08-03 04:00 AM    3207.89530000      268661.23137500
2024-08-03 05:00 AM    3238.30080000      271207.69200000
2024-08-03 06:00 AM    3212.89610000      269080.04837500
2024-08-03 07:00 AM    3259.45930000      272979.71637500
2024-08-03 08:00 AM    3237.72690000      271159.62787500
2024-08-03 09:00 AM    3225.55170000      270139.95487500
2024-08-03 10:00 AM    3214.34950000      269201.77062500
2024-08-03

In [14]:
if __name__ == '__main__':
    main()

Model Mean Squared Error: 0.00
Model Accuracy (R² score): 97.81%

Best time to buy DOGEUSDT: 2024-08-03 08:00 PM at price $0.11239810 (9.41 INR)
Best time to sell DOGEUSDT: 2024-08-06 06:00 PM at price $0.12435160 (10.41 INR)

Predicted Prices for DOGEUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-08-03 12:00 AM       0.11407300           9.55361375
2024-08-03 01:00 AM       0.11720830           9.81619512
2024-08-03 02:00 AM       0.11489440           9.62240600
2024-08-03 03:00 AM       0.11302550           9.46588562
2024-08-03 04:00 AM       0.11861940           9.93437475
2024-08-03 05:00 AM       0.11688740           9.78931975
2024-08-03 06:00 AM       0.11729570           9.82351487
2024-08-03 07:00 AM       0.11872580           9.94328575
2024-08-03 08:00 AM       0.11808300           9.88945125
2024-08-03 09:00 AM       0.11661820           9.76677425
2024-08-03 10:00 AM       0.11253920           9.42515800
2024-08-03 11:00 AM     

In [15]:
if __name__ == '__main__':
    main()

Model Mean Squared Error: 0.00
Model Accuracy (R² score): 99.37%

Best time to buy XRPUSDT: 2024-08-03 02:00 AM at price $0.54991800 (46.06 INR)
Best time to sell XRPUSDT: 2024-08-06 07:00 AM at price $0.61745200 (51.71 INR)

Predicted Prices for XRPUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-08-03 12:00 AM       0.56525600          47.34019000
2024-08-03 01:00 AM       0.57530900          48.18212875
2024-08-03 02:00 AM       0.54991800          46.05563250
2024-08-03 03:00 AM       0.56135700          47.01364875
2024-08-03 04:00 AM       0.59204700          49.58393625
2024-08-03 05:00 AM       0.58875800          49.30848250
2024-08-03 06:00 AM       0.58156600          48.70615250
2024-08-03 07:00 AM       0.59572400          49.89188500
2024-08-03 08:00 AM       0.58285200          48.81385500
2024-08-03 09:00 AM       0.57591000          48.23246250
2024-08-03 10:00 AM       0.57885000          48.47868750
2024-08-03 11:00 AM       

In [17]:
if __name__ == '__main__':
    main()

Model Mean Squared Error: 18.07
Model Accuracy (R² score): 97.88%

Best time to buy BNBUSDT: 2024-08-03 12:00 AM at price $562.11100000 (47076.80 INR)
Best time to sell BNBUSDT: 2024-08-03 06:00 AM at price $581.25700000 (48680.27 INR)

Predicted Prices for BNBUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-08-03 12:00 AM     562.11100000       47076.79625000
2024-08-03 01:00 AM     573.14200000       48000.64250000
2024-08-03 02:00 AM     578.94500000       48486.64375000
2024-08-03 03:00 AM     575.47800000       48196.28250000
2024-08-03 04:00 AM     574.78500000       48138.24375000
2024-08-03 05:00 AM     575.60400000       48206.83500000
2024-08-03 06:00 AM     581.25700000       48680.27375000
2024-08-03 07:00 AM     572.10400000       47913.71000000
2024-08-03 08:00 AM     576.63800000       48293.43250000
2024-08-03 09:00 AM     579.42100000       48526.50875000
2024-08-03 10:00 AM     577.56100000       48370.73375000
2024-08-03 11:0

In [18]:
if __name__ == '__main__':
    main()

Model Mean Squared Error: 0.00
Model Accuracy (R² score): 94.75%

Best time to buy USDCUSDT: 2024-08-03 01:00 AM at price $1.00025900 (83.77 INR)
Best time to sell USDCUSDT: 2024-08-03 09:00 PM at price $1.00121700 (83.85 INR)

Predicted Prices for USDCUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-08-03 12:00 AM       1.00086200          83.82219250
2024-08-03 01:00 AM       1.00025900          83.77169125
2024-08-03 02:00 AM       1.00099500          83.83333125
2024-08-03 03:00 AM       1.00095800          83.83023250
2024-08-03 04:00 AM       1.00121300          83.85158875
2024-08-03 05:00 AM       1.00114700          83.84606125
2024-08-03 06:00 AM       1.00095400          83.82989750
2024-08-03 07:00 AM       1.00114000          83.84547500
2024-08-03 08:00 AM       1.00116100          83.84723375
2024-08-03 09:00 AM       1.00109500          83.84170625
2024-08-03 10:00 AM       1.00111600          83.84346500
2024-08-03 11:00 AM    